In [1]:
!pip install ultralytics
!pip install opencv-python
!pip install matplotlib
!pip install tqdm
!pip install pillow
!pip install pyyaml
!pip install seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existin

In [3]:
import os
import glob
import json
import shutil
import random
import numpy as np
import torch
from tqdm import tqdm
from ultralytics import YOLO
from datetime import datetime
from PIL import Image

# BDD100K Dataset paths
BASE_DIR = "/kaggle/working/vehicle_pedestrian_detection_yolo"
ORIGINAL_DS_DIR = "/kaggle/input/bdd-dataset-100k"  # Base BDD100K directory
IMAGES_DIR = os.path.join(ORIGINAL_DS_DIR, "bdd100k", "images", "100k")
TRAIN_IMAGES = os.path.join(IMAGES_DIR, "train")
VAL_IMAGES = os.path.join(IMAGES_DIR, "val")
ANNOTATIONS_FILE = os.path.join(ORIGINAL_DS_DIR, "labels", "bdd100k_labels_images_train.json")
VAL_ANNOTATIONS_FILE = os.path.join(ORIGINAL_DS_DIR, "labels", "bdd100k_labels_images_val.json")

DATASET_DIR = os.path.join(BASE_DIR, "dataset")

EPOCHS = 30
BATCH_SIZE = 32
WORKERS = 8
IMG_SIZE = (640, 640)
YOLO_MODEL_SIZE = "yolov8l.pt"
CONF_THRESHOLD = 0.25

os.makedirs(os.path.join(DATASET_DIR, "images", "train"), exist_ok=True)
os.makedirs(os.path.join(DATASET_DIR, "images", "val"), exist_ok=True)
os.makedirs(os.path.join(DATASET_DIR, "labels", "train"), exist_ok=True)
os.makedirs(os.path.join(DATASET_DIR, "labels", "val"), exist_ok=True)

print(f"PyTorch CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    device = '0'
    os.environ['CUDA_MODULE_LOADING'] = 'LAZY'
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
else:
    print("No CUDA devices available, using CPU")
    device = 'cpu'

def load_bdd100k_annotations(annotation_file):
    print(f"Loading annotations from {annotation_file}")
    try:
        with open(annotation_file, 'r') as f:
            annotations = json.load(f)
        print(f"Loaded annotations for {len(annotations)} images")
        return annotations
    except Exception as e:
        print(f"Error loading annotations: {e}")
        return []

def extract_class_mapping(annotations):
    unique_categories = set()
    
    print("Scanning for unique object categories...")
    for img_anno in tqdm(annotations):
        for label in img_anno.get("labels", []):
            category = label.get("category")
            if category:
                unique_categories.add(category)
    
    sorted_categories = sorted(list(unique_categories))
    category_to_id = {category: i for i, category in enumerate(sorted_categories)}
    
    print(f"Found {len(sorted_categories)} unique categories: {sorted_categories}")
    return category_to_id, sorted_categories

def convert_bdd100k_to_yolo(image_file, annotations, category_to_id):
    img_name = os.path.basename(image_file)
    
    img_anno = None
    for anno in annotations:
        if anno.get("name") == img_name:
            img_anno = anno
            break
    
    if not img_anno:
        return []

    try:
        with Image.open(image_file) as img:
            img_width, img_height = img.size
    except Exception as e:
        print(f"Error reading image {image_file}: {e}")
        return []
    
    yolo_annotations = []
    
    for label in img_anno.get("labels", []):
        category = label.get("category")
        if category not in category_to_id:
            continue
        
        box = label.get("box2d")
        if not box:
            continue
        
        xmin = float(box["x1"])
        ymin = float(box["y1"])
        xmax = float(box["x2"])
        ymax = float(box["y2"])
        
        x_center = (xmin + xmax) / 2 / img_width
        y_center = (ymin + ymax) / 2 / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height
        
        if width <= 0 or height <= 0 or not all(0 <= val <= 1 for val in [x_center, y_center, width, height]):
            continue
        
        class_id = category_to_id[category]
        yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")
    
    return yolo_annotations

def find_all_training_images():
    all_images = []
    
    subdirs = [d for d in os.listdir(TRAIN_IMAGES) if os.path.isdir(os.path.join(TRAIN_IMAGES, d))]
    
    if subdirs and subdirs[0].startswith('train'):
        print(f"Found training subdirectories: {subdirs}")
        for subdir in subdirs:
            subdir_path = os.path.join(TRAIN_IMAGES, subdir)
            for ext in ['*.jpg', '*.jpeg', '*.png']:
                all_images.extend(glob.glob(os.path.join(subdir_path, ext)))
    else:
        for ext in ['*.jpg', '*.jpeg', '*.png']:
            all_images.extend(glob.glob(os.path.join(TRAIN_IMAGES, ext)))
    
    print(f"Found {len(all_images)} training images")
    return all_images

def find_all_validation_images():
    all_images = []
    
    for ext in ['*.jpg', '*.jpeg', '*.png']:
        all_images.extend(glob.glob(os.path.join(VAL_IMAGES, ext)))
    
    print(f"Found {len(all_images)} validation images")
    return all_images

def prepare_dataset():
    print("Loading annotations...")
    train_annotations = load_bdd100k_annotations(ANNOTATIONS_FILE)
    val_annotations = load_bdd100k_annotations(VAL_ANNOTATIONS_FILE)
    
    if not train_annotations:
        print("ERROR: No training annotations found!")
        return None
    
    print("Extracting class mapping...")
    category_to_id, class_names = extract_class_mapping(train_annotations + val_annotations)
    
    print("Finding training images...")
    train_images = find_all_training_images()
    
    print("Finding validation images...")
    val_images = find_all_validation_images()
    
    print("Processing training set...")
    train_with_labels = 0
    for img_path in tqdm(train_images):
        img_name = os.path.basename(img_path)
        dest_img = os.path.join(DATASET_DIR, "images", "train", img_name)
        
        shutil.copy2(img_path, dest_img)
        
        yolo_annotations = convert_bdd100k_to_yolo(img_path, train_annotations, category_to_id)
        if yolo_annotations:
            label_name = os.path.splitext(img_name)[0] + ".txt"
            label_path = os.path.join(DATASET_DIR, "labels", "train", label_name)
            with open(label_path, 'w') as f:
                f.write("\n".join(yolo_annotations))
            train_with_labels += 1
    
    print("Processing validation set...")
    val_with_labels = 0
    for img_path in tqdm(val_images):
        img_name = os.path.basename(img_path)
        dest_img = os.path.join(DATASET_DIR, "images", "val", img_name)
        
        shutil.copy2(img_path, dest_img)
        
        yolo_annotations = convert_bdd100k_to_yolo(img_path, val_annotations, category_to_id)
        if yolo_annotations:
            label_name = os.path.splitext(img_name)[0] + ".txt"
            label_path = os.path.join(DATASET_DIR, "labels", "val", label_name)
            with open(label_path, 'w') as f:
                f.write("\n".join(yolo_annotations))
            val_with_labels += 1
    
    print("Creating dataset.yaml...")
    dataset_yaml_path = os.path.join(DATASET_DIR, "dataset.yaml")
    with open(dataset_yaml_path, "w") as f:
        f.write(f"train: ./images/train\n")
        f.write(f"val: ./images/val\n")
        f.write(f"nc: {len(class_names)}\n")
        f.write(f"names: {class_names}\n")
    
    train_imgs = glob.glob(os.path.join(DATASET_DIR, "images", "train", "*.*"))
    val_imgs = glob.glob(os.path.join(DATASET_DIR, "images", "val", "*.*"))
    train_labels = glob.glob(os.path.join(DATASET_DIR, "labels", "train", "*.txt"))
    val_labels = glob.glob(os.path.join(DATASET_DIR, "labels", "val", "*.txt"))
    
    print("\nDataset prepared:")
    print(f"  Training images: {len(train_imgs)} ({train_with_labels} with labels)")
    print(f"  Training labels: {len(train_labels)}")
    print(f"  Validation images: {len(val_imgs)} ({val_with_labels} with labels)")
    print(f"  Validation labels: {len(val_labels)}")
    print(f"  Categories: {len(class_names)}")
    print(f"  Classes: {', '.join(class_names)}")
    
    return dataset_yaml_path

def train_model(dataset_yaml_path):
    print("\nInitializing YOLOv8 model...")
    try:
        model = YOLO(YOLO_MODEL_SIZE)
        print("Model loaded successfully")
    except Exception as e:
        print(f"Error loading model: {e}")
        print("Downloading model using torch hub...")
        torch.hub.load('ultralytics/yolov8', 'yolov8l', pretrained=True)
        model = YOLO(YOLO_MODEL_SIZE)
    
    print("\nStarting training...")
    os.chdir(DATASET_DIR)
    
    results = model.train(
        data=dataset_yaml_path,
        epochs=EPOCHS,
        workers=WORKERS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        name='yolo_bdd100k_detector',
        patience=15,
        save=True,
        device=device,
        cache=False,
        amp=True,
        rect=True,
        plots=True,
        augment=True,
        close_mosaic=10,
        overlap_mask=True,
        cos_lr=True,
        pretrained=True,
        seed=42,
        profile=True,
        verbose=True,
        mosaic=0.8,
        mixup=0.1,
        copy_paste=0.1,
        degrees=10.0,
        scale=0.5,
        save_period=10
    )
    
    print("\nValidating trained model...")
    best_model_path = os.path.join("runs", "detect", "yolo_bdd100k_detector", "weights", "best.pt")
    
    metrics = model.val(
        data=dataset_yaml_path,
        conf=CONF_THRESHOLD
    )
    
    kaggle_output_path = "/kaggle/working/bdd100k_detector_best.pt"
    
    print("\nEvaluating model performance...")
    evaluate_detection_model(
        model_path=best_model_path,
        data_yaml=dataset_yaml_path,
        conf_threshold=CONF_THRESHOLD,
        iou_threshold=0.5,
        save_dir="/kaggle/working/detection_metrics"
    )
    
    shutil.copy2(best_model_path, kaggle_output_path)
    print(f"Model saved to Kaggle working directory: {kaggle_output_path}")
    
    # Save the results images
    results_img = f"runs/detect/yolo_bdd100k_detector/results.png"
    confusion_matrix = f"runs/detect/yolo_bdd100k_detector/confusion_matrix.png"
    
    if os.path.exists(results_img):
        shutil.copy2(results_img, "/kaggle/working/results.png")
    if os.path.exists(confusion_matrix):
        shutil.copy2(confusion_matrix, "/kaggle/working/confusion_matrix.png")
    
    print("\nTraining and evaluation complete!")

def evaluate_detection_model(model_path, data_yaml, conf_threshold=0.25, iou_threshold=0.5, save_dir="metrics"):
    os.makedirs(save_dir, exist_ok=True)
    
    model = YOLO(model_path)
    
    metrics = model.val(
        data=data_yaml,
        conf=conf_threshold,
        iou=iou_threshold,
        verbose=True,
        save_json=True,
        save_hybrid=True,
        plots=True
    )
    
    results = {
        "mAP50": float(metrics.box.map50),
        "mAP50-95": float(metrics.box.map),
        "precision": float(metrics.box.mp),
        "recall": float(metrics.box.mr),
        "f1": float(2 * metrics.box.mp * metrics.box.mr / (metrics.box.mp + metrics.box.mr + 1e-10)),
        "conf_threshold": conf_threshold,
        "iou_threshold": iou_threshold,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    
    per_class_ap = {}
    if hasattr(metrics.box, 'ap_class_index') and hasattr(metrics.box, 'ap50'):
        class_names = model.names
        for i, class_idx in enumerate(metrics.box.ap_class_index):
            class_name = class_names[int(class_idx)]
            per_class_ap[class_name] = float(metrics.box.ap50[i])
        results["per_class_ap50"] = per_class_ap
    
    with open(os.path.join(save_dir, f"metrics_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"), 'w') as f:
        json.dump(results, f, indent=4)
    
    print("\n===== BDD100K DETECTION MODEL EVALUATION =====")
    print(f"mAP@0.5: {results['mAP50']:.4f}")
    print(f"mAP@0.5:0.95: {results['mAP50-95']:.4f}")
    print(f"Precision: {results['precision']:.4f}")
    print(f"Recall: {results['recall']:.4f}")
    print(f"F1 Score: {results['f1']:.4f}")
    
    if "per_class_ap50" in results:
        print("\nPer-class AP@0.5:")
        for class_name, ap in results["per_class_ap50"].items():
            print(f"  {class_name}: {ap:.4f}")
    
    return results

if __name__ == "__main__":
    dataset_yaml_path = prepare_dataset()
    if dataset_yaml_path:
        train_model(dataset_yaml_path)
    else:
        print("Dataset preparation failed. Exiting.")

PyTorch CUDA available: True
GPU: Tesla P100-PCIE-16GB
Loading annotations...
Loading annotations from /kaggle/input/bdd-dataset-100k/labels/bdd100k_labels_images_train.json
Loaded annotations for 69863 images
Loading annotations from /kaggle/input/bdd-dataset-100k/labels/bdd100k_labels_images_val.json
Loaded annotations for 10000 images
Extracting class mapping...
Scanning for unique object categories...


100%|██████████| 79863/79863 [00:00<00:00, 226664.44it/s]


Found 12 unique categories: ['bike', 'bus', 'car', 'drivable area', 'lane', 'motor', 'person', 'rider', 'traffic light', 'traffic sign', 'train', 'truck']
Finding training images...
Found 1156 training images
Finding validation images...
Found 10000 validation images
Processing training set...


100%|██████████| 1156/1156 [00:16<00:00, 69.44it/s]


Processing validation set...


...

Speed: 0.1ms preprocess, 26.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/yolo_bdd100k_detector2

Evaluating model performance...
WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in in the future.
Ultralytics 8.3.136 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 112 layers, 43,615,860 parameters, 0 gradients, 164.9 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1067.9±275.3 MB/s, size: 43.4 KB)


val: Scanning /kaggle/working/vehicle_pedestrian_detection_yolo/dataset/labels/val.cache... 10000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10000/10000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 625/625 [02:53<00:00,  3.60it/s]


                   all      10000     185526       0.51      0.354      0.437      0.252
                  bike        578       1007      0.493      0.299      0.406        0.2
                   bus       1242       1597      0.391      0.429       0.39      0.302
                   car       9879     102506      0.783      0.644      0.753      0.505
                 motor        334        452      0.401      0.175      0.277      0.136
                person       3220      13262      0.698      0.477      0.615       0.33
                 rider        515        649      0.475      0.265      0.368      0.165
         traffic light       5653      26885      0.691      0.402      0.548      0.225
          traffic sign       8221      34908      0.633      0.483      0.571       0.33
                 train         14         15          0          0          0          0
                 truck       2689       4245      0.538      0.363      0.442      0.325


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 10.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Saving runs/detect/val/predictions.json...
Results saved to runs/detect/val

===== BDD100K DETECTION MODEL EVALUATION =====
mAP@0.5: 0.4369
mAP@0.5:0.95: 0.2518
Precision: 0.5104
Recall: 0.3537
F1 Score: 0.4178

Per-class AP@0.5:
  bike: 0.4062
  bus: 0.3898
  car: 0.7532
  motor: 0.2769
  person: 0.6153
  rider: 0.3675
  traffic light: 0.5476
  traffic sign: 0.5706
  train: 0.0000
  truck: 0.4419
Model saved to Kaggle working directory: /kaggle/working/bdd100k_detector_best.pt

Training and evaluation complete!


In [4]:
!zip -r /kaggle/working/output.zip /kaggle/working/vehicle_pedestrian_detection_yolo/dataset/runs/detect

  adding: kaggle/working/vehicle_pedestrian_detection_yolo/dataset/runs/detect/ (stored 0%)
  adding: kaggle/working/vehicle_pedestrian_detection_yolo/dataset/runs/detect/val/ (stored 0%)
  adding: kaggle/working/vehicle_pedestrian_detection_yolo/dataset/runs/detect/val/F1_curve.png (deflated 10%)
  adding: kaggle/working/vehicle_pedestrian_detection_yolo/dataset/runs/detect/val/R_curve.png (deflated 12%)
  adding: kaggle/working/vehicle_pedestrian_detection_yolo/dataset/runs/detect/val/val_batch2_pred.jpg (deflated 6%)
  adding: kaggle/working/vehicle_pedestrian_detection_yolo/dataset/runs/detect/val/predictions.json (deflated 79%)
  adding: kaggle/working/vehicle_pedestrian_detection_yolo/dataset/runs/detect/val/P_curve.png (deflated 7%)
  adding: kaggle/working/vehicle_pedestrian_detection_yolo/dataset/runs/detect/val/PR_curve.png (deflated 6%)
  adding: kaggle/working/vehicle_pedestrian_detection_yolo/dataset/runs/detect/val/val_batch0_labels.jpg (deflated 8%)
  adding: kaggle/work